In [ ]:
# all plots will be set directly below the code cell that produced it
%matplotlib inline
from matplotlib import pyplot as plt
from IPython.display import clear_output
import scipy.ndimage
# remove grid lines
import numpy as np
import time
import cv2

## Helper functions

In [ ]:
# function to read and resize an image
def read_and_resize(filename, grayscale = False, fx= 0.5, fy=0.5):
    if grayscale:
      img_result = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
    else:
      imgbgr = cv2.imread(filename, cv2.IMREAD_COLOR)
      # convert to rgb
      img_result = cv2.cvtColor(imgbgr, cv2.COLOR_BGR2RGB)
    # resize
    img_result = cv2.resize(img_result, None, fx=fx, fy=fy, interpolation = cv2.INTER_CUBIC)
    return img_result
 
    
def showInRow(list_of_images, titles = None, disable_ticks = False):
  count = len(list_of_images)
  for idx in range(count):
    # set inline plots size
    plt.rcParams["figure.figsize"] = (16, 10) # (w, h)
    subplot = plt.subplot(1, count, idx+1)
    if titles is not None:
      subplot.set_title(titles[idx])
      
    img = list_of_images[idx]
    cmap = 'gray' if (len(img.shape) == 2 or img.shape[2] == 1) else None
    subplot.imshow(img, cmap=cmap)
    if disable_ticks:
      plt.xticks([]), plt.yticks([])
  plt.show()

## Harris corner detector

In [ ]:
Cylindertop = read_and_resize('Cylindertop.jpg')
 
gray = cv2.cvtColor(Cylindertop,cv2.COLOR_RGB2GRAY)
 
# cv.cornerHarris(src, blockSize, ksize, k)
# blockSize Neighborhood size.
# ksize Aperture parameter for the Sobel operator.
corners = cv2.cornerHarris(gray,3,3,0.08)
 
# Dilate responses to make them more visible (only needed for visualization!)
dest = cv2.dilate(corners, None) 
  
# Color pixels in the original image red, where
# the Harris response is above a given threshold
Cylindertop[dest > 0.04 * dest.max()]=[255, 0, 0] 
 
    
showInRow([corners, dest, Cylindertop])

## Object detection using sift

In [ ]:
book = read_and_resize('Cylindertop.jpg', 0)
scene = read_and_resize('All1.jpg', 0)
book = book[:,0:1500]

MIN_MATCH_COUNT = 5
 
sift = cv2.xfeatures2d.SIFT_create()
# find the keypoints and descriptors with SIFT
kp1, des1 = sift.detectAndCompute(book,None)
kp2, des2 = sift.detectAndCompute(scene,None)
 
 
# BFMatcher with default params
bf = cv2.BFMatcher()
matches = bf.knnMatch(des1,des2, k=2)
 
# Apply ratio test
good = []
for m,n in matches:
    if m.distance < 0.75*n.distance:
        good.append(m)
 
 
if len(good)>MIN_MATCH_COUNT:
    src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
    dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)
 
    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,7.0)
    # findHomography is a method of finding the relation between two images of the same object taken from different viewing directions of the camera
    # findHomography returns a mask that tells us, which point pairs did not pass the Random Sample Consensus (RANSAC) filter
    matchesMask = mask.ravel().tolist()
 
    h,w,c = book.shape
    # Take corners of the first image and transform them onto the second image
    pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
    dst = cv2.perspectiveTransform(pts,M)
 
    scene = cv2.polylines(scene,[np.int32(dst)],True,255,3, cv2.LINE_AA)
 
else:
    print ("Not enough matches were found, only %d. Needed: %d" % (len(good),MIN_MATCH_COUNT))
    matchesMask = None
# # Find the homography using the detected matches
# < your code here >
        
# # Use the homography to stitch the images together
# < your code here >
    
draw_params = dict(matchColor = (0,255,255), # draw matches in green color
                   singlePointColor = (0,255,0),
                   matchesMask = matchesMask, # draw only inliers
                   flags = 4)

 
img3 = cv2.drawMatches(book,kp1,scene,kp2,good,None,**draw_params)
 
showInRow([ img3])

sift_f(obj, scene)## Real time object detection

In [ ]:
def sift_f(obj, scene):
  sift = cv2.xfeatures2d.SIFT_create()
  # find the keypoints and descriptors with SIFT
  kp1, des1 = sift.detectAndCompute(obj,None)
  kp2, des2 = sift.detectAndCompute(scene,None)
 
 
  # BFMatcher with default params
  bf = cv2.BFMatcher()
  matches = bf.knnMatch(des1,des2, k=2)
 
  # Apply ratio test
  good = []
  for m,n in matches:
      if m.distance < 0.7*n.distance:
          good.append(m)
  
  MIN_MATCH_COUNT=4
 
  if(len(good)>=MIN_MATCH_COUNT):
    tp=[]
    qp=[]
 
    for m in good:
        tp.append(kp1[m.queryIdx].pt)
        qp.append(kp2[m.trainIdx].pt)
 
    tp,qp=np.float32((tp,qp))
 
    H,status=cv2.findHomography(tp,qp,cv2.RANSAC,3.0)
 
    h,w,c=obj.shape
    trainBorder=np.float32([[[0,0],[0,h-1],[w-1,h-1],[w-1,0]]])
    queryBorder=cv2.perspectiveTransform(trainBorder,H)
    cv2.polylines(scene,[np.int32(queryBorder)],True,(0,255,0),5)
    return scene
  else:
    print ("Not Enough matches found- %d/%d"%(len(good),MIN_MATCH_COUNT))

In [20]:
book = read_and_resize('Cylindertop.jpg', 0)
obj = book[:,0:1500]

cap = cv2.VideoCapture(0)

# # Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi',fourcc, 20.0, (640,480))

while(cap.isOpened()):
    ret, frame = cap.read()
    if ret==True:
        frame = cv2.flip(frame,1)
        cv2.waitkey(10)
        # write the flipped frame
        scene = frame
        frame2 = sift_f(obj, scene)
        out.write(frame2)
        print(frame2)
        print()
        print(scene)
        cv2.imshow('frame',frame2)
#         showInRow([scene  , obj])
    

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break 
    else:
        break
# Release everything if job is finished
cap.release()
out.release()
cv2.destroyAllWindows()



# cap = cv2.VideoCapture(0)
# ret, frame = cap.read()
# if ret==True:
#     frame = cv2.flip(frame,1)

    
# scene = frame
# sift_f(obj, scene)
# # scene = cv2.imshow('frame',frame)
# showInRow([scene  , obj])


In [ ]:




sift_f(obj, scene)

## References

harris corner detector [link](https://muthu.co/harris-corner-detector-implementation-in-python/)

SIFT [link](https://docs.opencv.org/trunk/da/df5/tutorial_py_sift_intro.html)

SIFT Python Implementation [link](https://medium.com/@lerner98/implementing-sift-in-python-36c619df7945)

Panoramas [link](https://www.pyimagesearch.com/2016/01/11/opencv-panorama-stitching/)

Object detection with SIFT [link](https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_feature2d/py_feature_homography/py_feature_homography.html)